## The Score-P Python Kernel
This is the Score-P Python Kernel that allows you to execute Jupyter Notebooks with Score-P for performance analysis. It supports the usual Jupyter interactivity between cells though with some limitations (see **General Limitations**).

The kernel requires [Score-P](https://www.vi-hps.org/projects/score-p/) and [Score-P Python bindings](https://github.com/score-p/scorep_binding_python) to be installed.

### Setup
You can set up your Score-P environment by executing a cell that starts with the `%%scorep_env magic command`.

You can set the Score-P Python binding arguments by executing a cell that starts with `%%scorep_python_binding_arguments`.

In [ ]:
%%scorep_env
SCOREP_ENABLE_TRACING=1
SCOREP_ENABLE_PROFILING=0
SCOREP_TOTAL_MEMORY=3g

In [ ]:
%%scorep_python_binding_arguments
--noinstrumenter

### Cells instrumentation

Cells that should be executed with Score-P have to be marked with `%%execute_with_scorep` in the first line. Cells without that command are executed as ordinary Python processes.

In [ ]:
%%execute_with_scorep
import scorep
class A:
    desc = "This class and method should be..."
    def print_desc(self, x):
        print(self.desc + str(x))

a = A()

In [ ]:
a.print_desc("known here")

In [ ]:
a.desc = "new desc"

In [ ]:
print(a.desc)

In [ ]:
%%execute_with_scorep
import scorep
with scorep.instrumenter.enable():
    a.desc = "new desc2"

In [ ]:
print(a.desc)

In [ ]:
%%execute_with_scorep
import scorep
import time

def sleep_and_double(x):
    time.sleep(x)
    return 2*x

with scorep.instrumenter.enable():
    x = 5
    x = sleep_and_double(x)
    x = sleep_and_double(x)

In [ ]:
print(x)

### Multicell mode
You can also treat multiple cells as one single cell by using the multicell mode.

For that, you can mark the cells in the order you wish to execute them. Start the marking process by a cell that starts with the `%%enable_multicellmode` command.

Now mark your cells by running them. Note that the cells will not be executed at this point but will be marked for later execution.
You can stop the marking and execute all the marked cells by running a cell that starts with `%%finalize_multicellmode` command.
This will execute all the marked cells orderly with Score-P. Note that the `%%execute_with_scorep` command has no effect in the multi cell mode.

There is no "unmark" command available but you can abort the multicellmode by the `%%abort_multicellmode` command. Start your marking process again if you have marked your cells in the wrong order.

The `%%enable_multicellmode`, `%%finalize_multicellmode` and `%%abort_multicellmode` commands should be run in an exclusive cell. Additional code in the cell will be ignored.

In [ ]:
%%enable_multicellmode

In [ ]:
with scorep.instrumenter.enable():
    class B:
        desc = "This is a class defined in multi cell mode"
        def print_desc(self, x):
            print(self.desc + str(x))

In [ ]:
import scorep
with scorep.instrumenter.enable():
    b = B()

In [ ]:
with scorep.instrumenter.enable():
    b.print_desc("...and this object is initialized and used in it.")

In [ ]:
b.desc = "modified desc"

In [ ]:
print(b.desc)

In [ ]:
%%finalize_multicellmode

### Write mode

With write mode you can convert notebook cells into Python script which is then to be executed by Score-P bindings using auxillary bash script. 

Similarly to multicell mode, you can run a cell with `%%start_writefile` magic command to enable write mode. Then, running the cells will record their contents instead of executing them. Environment variables and Score-P Python bindings arguments will be written to bash script. Finish the write mode with `%%end_writefile` cell.

You can specify Python script name by providing it as an argument for `%%start_writefile`.

In [ ]:
%%start_writefile myscript.py

In [ ]:
%%scorep_env
SCOREP_ENABLE_TRACING=1
SCOREP_ENABLE_PROFILING=0
SCOREP_TOTAL_MEMORY=3g

In [ ]:
%%scorep_python_binding_arguments
--noinstrumenter

In [ ]:
print("Cell without instrumentation.")

In [ ]:
%%execute_with_scorep

import numpy as np
import scorep

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = a.dot(b)

In [ ]:
%%enable_multicellmode

with scorep.instrumenter.enable():
    d = a.outer(b)

In [ ]:
with scorep.instrumenter.enable():
    e = b.outer(a)

In [ ]:
%%abort_multicellmode

In [ ]:
%%finalize_multicellmode

In [ ]:
%%end_writefile

You can now run `myscript_run.sh` to execute Python script with Score-P bindings.